In [ ]:
%pip uninstall cupy-cuda12x cupy -y
%pip install cupy-cuda12x --no-cache-dir

In [ ]:
import os
import glob

cuda_bin = r"C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.6\bin"
nvrtc_dlls = glob.glob(os.path.join(cuda_bin, "nvrtc*.dll"))

print("Found NVRTC DLLs:")
for dll in nvrtc_dlls:
    print(f"  - {os.path.basename(dll)}")

In [ ]:
import os

# Add both bin and lib folders
cuda_base = r"C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.6"
cuda_bin = os.path.join(cuda_base, "bin")
cuda_lib = os.path.join(cuda_base, "lib", "x64")

# Add to PATH
os.environ['PATH'] = cuda_bin + os.pathsep + cuda_lib + os.pathsep + os.environ['PATH']
print(f"✓ Added CUDA bin and lib to PATH")

# Also set CUDA_PATH
os.environ['CUDA_PATH'] = cuda_base
print(f"✓ Set CUDA_PATH")

# Now try CuPy
import cupy as cp
x = cp.random.random((1000, 1000))
y = cp.sum(x)
print(f"✓ GPU computation successful: {float(y):.2f}")

In [ ]:
import cupy as cp
print(cp.__version__)  # Should print: 13.6.0
print(cp.cuda.runtime.getDeviceCount())  # Should show your GPU count

In [ ]:
import subprocess
try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
    print(result.stdout)
except FileNotFoundError:
    print("❌ nvidia-smi not found - checking if you have NVIDIA GPU...")

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
import multiprocessing

# Optional GPU acceleration (CuPy); set use_gpu=True below to attempt
try:
    import cupy as cp  # Use the installed binary
    _has_cupy = True
except ImportError:
    try:
        import cupy as cp  # Fallback
        _has_cupy = True
    except ImportError:
        cp = None
        _has_cupy = False

# Global variables for worker processes
_returns_matrix = None
_ordered_cols = None
_years = None
_block_size = None

def init_worker(returns_mat, cols, yrs, blk_sz):
    """Initialize each worker process with shared data"""
    global _returns_matrix, _ordered_cols, _years, _block_size
    _returns_matrix = returns_mat
    _ordered_cols = cols
    _years = yrs
    _block_size = blk_sz

# 1. Setup & Data Fetching
# Tickers ranked by performance metrics (from best to worst)
tickers = ['USMV', 'SCHD', 'VDC', 'VOO', 'QUAL', 'SPMO', 'AVDV', 'XLP',
           'VUG', 'XLV', 'VTI', 'AVUV', 'VXUS', 'QQQM', 'SPGP', 'VT',
           'JQUA', 'DYNF', 'AVDE', 'AVEM']

# Download data
raw_data = yf.download(tickers, start="2020-01-01", end="2025-01-01")

# Handle Adj Close fallback - use 'Adj Close' if available, otherwise 'Close'
if 'Adj Close' in raw_data.columns.get_level_values(0):
    data = raw_data['Adj Close']
else:
    # If multi-level columns but no Adj Close
    if isinstance(raw_data.columns, pd.MultiIndex):
        if 'Close' in raw_data.columns.get_level_values(0):
            data = raw_data['Close']
        else:
            raise ValueError("Neither 'Adj Close' nor 'Close' found in data")
    else:
        data = raw_data

returns = data.pct_change().dropna()

# 2. Define Your Two Portfolios
# Portfolio A {'SPMO': 0.1, 'VTI': 0.3, 'VONG': 0.1, 'VXUS': 0.075, 'AVUV': 0.1, 'AVDV': 0.075, 'JQUA': 0.1, 'DYNF': 0.15}
weights_a = {'SPMO': 0.1, 'VTI': 0.4, 'AVUV': 0.1, 'AVDV': 0.1, 'DYNF': 0.2, 'AVDE': 0.075, 'AVEM': 0.025}
# Portfolio B
weights_b = {'VTI': 0.85, 'VXUS': 0.15}

# Simulation controls
n_sims = 10000  # increase/decrease to control number of Monte Carlo paths
block_size = 20
years = 30
year_streak = 5
use_gpu = False  # set True to attempt CuPy GPU path (serial execution)

# Optimal worker count: 4-6 workers is the sweet spot for most systems
max_workers = 10  # Conservative, reliable setting
use_processes = False  # True = ProcessPoolExecutor (separate processes)
use_threads = True # True = ThreadPoolExecutor (threads in same process)

# Precompute structures for faster simulation
ordered_cols = list(returns.columns)
returns_matrix_np = returns.to_numpy()

# Choose array module
xp = cp if (use_gpu and _has_cupy) else np
if use_gpu and not _has_cupy:
    print("CuPy not available; falling back to CPU (NumPy).")

# Build weight vectors aligned to returns columns
def build_weight_vector(weights, xp_mod):
    return xp_mod.array([weights.get(col, 0.0) for col in ordered_cols], dtype=float)

weights_a_vec = build_weight_vector(weights_a, xp)
weights_b_vec = build_weight_vector(weights_b, xp)

# If using GPU, move returns to device once
returns_matrix_gpu = xp.asarray(returns_matrix_np) if (xp is cp) else None

def simulate_future(returns_matrix, weights_vec, years=30, block_size=20, xp_mod=np):
    """Creates a 'Frankenstein' future by stitching random blocks (CPU or GPU)."""
    n_days = years * 252
    sim_blocks = []
    total_len = 0

    while total_len < n_days:
        start_idx = int(xp_mod.random.randint(0, returns_matrix.shape[0] - block_size))
        block = returns_matrix[start_idx : start_idx + block_size]
        port_block = xp_mod.sum(block * weights_vec, axis=1)
        sim_blocks.append(port_block)
        total_len += block_size

    sim_returns = xp_mod.concatenate(sim_blocks)[:n_days]
    wealth_index = xp_mod.cumprod(1 + sim_returns)
    # Move back to CPU numpy for downstream pandas/metrics
    wealth_index_cpu = wealth_index.get() if xp_mod is cp else wealth_index
    return pd.Series(wealth_index_cpu)

def calculate_metrics(path):
    """Calculate comprehensive metrics for a given path."""
    # Max Drawdown
    dd = (path / path.cummax() - 1).min()

    # Drawdown area (days below cummax times magnitude)
    drawdowns = path / path.cummax() - 1
    drawdown_area = (drawdowns[drawdowns < 0]).abs().sum()

    # Ulcer Index (sqrt of average squared drawdowns)
    ulcer_index = np.sqrt((drawdowns[drawdowns < 0] ** 2).sum() / len(path)) * 100

    # Final Growth
    growth = path.iloc[-1]

    # CAGR (years assumed)
    cagr = (growth ** (1 / years)) - 1

    # Daily returns volatility
    daily_returns = path.pct_change().dropna()
    volatility_annual = daily_returns.std() * np.sqrt(252)

    # Sharpe Ratio (assuming 2.5% risk-free rate)
    risk_free_rate = 0.025
    sharpe_ratio = (cagr - risk_free_rate) / volatility_annual if volatility_annual > 0 else 0

    # Sortino Ratio (only downside volatility)
    downside_returns = daily_returns[daily_returns < 0]
    downside_volatility = downside_returns.std() * np.sqrt(252) if len(downside_returns) > 0 else 0
    sortino_ratio = (cagr - risk_free_rate) / downside_volatility if downside_volatility > 0 else 0

    # Win Rate (% of days with positive returns)
    win_rate = (daily_returns > 0).sum() / len(daily_returns) * 100 if len(daily_returns) > 0 else 0

    # Positive Years % - count how many years had positive returns
    yearly_returns = []
    for year_idx in range(years):
        start_idx = year_idx * 252
        end_idx = (year_idx + 1) * 252
        if end_idx < len(path):
            year_return = (path.iloc[end_idx] / path.iloc[start_idx]) - 1
            yearly_returns.append(year_return)
    positive_years_pct = (sum(1 for r in yearly_returns if r > 0) / len(yearly_returns) * 100) if yearly_returns else 0

    # Recovery Factor = Total Return / Max Drawdown magnitude
    recovery_factor = (growth - 1) / abs(dd) if dd != 0 else 0

    # Decade CAGRs
    decade_cagrs = []
    for decade_start in [0, 10, 20]:
        start_idx = decade_start * 252
        end_idx = min((decade_start + 10) * 252, len(path) - 1)
        if end_idx > start_idx and start_idx < len(path) and end_idx < len(path):
            decade_growth = path.iloc[end_idx] / path.iloc[start_idx]
            decade_cagr = (decade_growth ** (1 / 10)) - 1 if decade_growth > 0 else 0
            decade_cagrs.append(decade_cagr)
        else:
            decade_cagrs.append(0)

    # Martin Ratio = Return / Ulcer Index
    martin_ratio = (growth - 1) / ulcer_index if ulcer_index > 0 else 0

    # Compounding Efficiency Score = Total Return / Drawdown Area
    efficiency = (growth - 1) / drawdown_area if drawdown_area > 0 else 0

    # Check for x-year underwater streak (x-years * 252)
    underwater = (path < path.cummax()).astype(int)
    streaks = []
    current_streak = 0
    for is_underwater in underwater:
        if is_underwater:
            current_streak += 1
        else:
            if current_streak > 0:
                streaks.append(current_streak)
            current_streak = 0
    if current_streak > 0:
        streaks.append(current_streak)

    max_streak = max(streaks) if streaks else 0
    has_xyr_streak = max_streak >= 252 * year_streak  # x years * 252 trading days

    # Total underwater days
    total_underwater_days = (underwater == 1).sum()

    return {
        'DD': dd,
        'Growth': growth,
        'CAGR': cagr,
        'Volatility_Annual': volatility_annual,
        'Sharpe_Ratio': sharpe_ratio,
        'Sortino_Ratio': sortino_ratio,
        'Win_Rate': win_rate,
        'Positive_Years_Pct': positive_years_pct,
        'Recovery_Factor': recovery_factor,
        'Decade1_CAGR': decade_cagrs[0] if len(decade_cagrs) > 0 else 0,
        'Decade2_CAGR': decade_cagrs[1] if len(decade_cagrs) > 1 else 0,
        'Decade3_CAGR': decade_cagrs[2] if len(decade_cagrs) > 2 else 0,
        'Drawdown_Area': drawdown_area,
        'Ulcer_Index': ulcer_index,
        'Martin_Ratio': martin_ratio,
        'Efficiency_Score': efficiency,
        f'Has_{year_streak}Yr_Streak': has_xyr_streak,
        'Max_Underwater_Streak_Days': max_streak,
        'Total_Underwater_Days': total_underwater_days
    }

# Helper for parallel execution
def _run_single_sim_cpu(weights_dict):
    """Run single simulation in worker process using global data"""
    global _returns_matrix, _ordered_cols, _years, _block_size
    weights_vec = np.array([weights_dict.get(col, 0.0) for col in _ordered_cols], dtype=float)
    path = simulate_future(_returns_matrix, weights_vec, years=_years, block_size=_block_size, xp_mod=np)
    return calculate_metrics(path)

# Execute simulations (CPU threading/multiprocessing or single-process GPU)
if __name__ == '__main__':
    if xp is cp:
        print("Running on GPU (serial execution)...")
        results_a = [calculate_metrics(simulate_future(returns_matrix_gpu, weights_a_vec, years=years, block_size=block_size, xp_mod=cp)) for _ in range(n_sims)]
        results_b = [calculate_metrics(simulate_future(returns_matrix_gpu, weights_b_vec, years=years, block_size=block_size, xp_mod=cp)) for _ in range(n_sims)]
        print(f"✓ GPU execution completed ({len(results_a)} simulations).")
    else:
        # CPU path
        if use_processes:
            # Calculate optimal chunksize: larger chunks = less overhead
            optimal_chunksize = max(n_sims // (4 * max_workers), 100)
            
            print(f"Running with ProcessPoolExecutor ({max_workers} workers, chunksize={optimal_chunksize})...")
            
            with ProcessPoolExecutor(
                max_workers=max_workers,
                mp_context=multiprocessing.get_context("spawn"),
                initializer=init_worker,
                initargs=(returns_matrix_np, ordered_cols, years, block_size)
            ) as executor:
                results_a = list(executor.map(_run_single_sim_cpu, [weights_a] * n_sims, 
                                             chunksize=optimal_chunksize))
                results_b = list(executor.map(_run_single_sim_cpu, [weights_b] * n_sims,
                                             chunksize=optimal_chunksize))
            
            print(f"✓ Multiprocessing completed ({len(results_a)} simulations).")
        elif use_threads:
            print(f"Running with ThreadPoolExecutor ({max_workers or multiprocessing.cpu_count()} threads)...")
            weights_a_vec_np = np.array([weights_a.get(col, 0.0) for col in ordered_cols], dtype=float)
            weights_b_vec_np = np.array([weights_b.get(col, 0.0) for col in ordered_cols], dtype=float)
            
            def _run_single_sim_cpu_thread(weights_vec_np):
                path = simulate_future(returns_matrix_np, weights_vec_np, years=years, block_size=block_size, xp_mod=np)
                return calculate_metrics(path)
            
            with ThreadPoolExecutor(max_workers=max_workers or multiprocessing.cpu_count()) as executor:
                results_a = list(executor.map(_run_single_sim_cpu_thread, [weights_a_vec_np] * n_sims))
                results_b = list(executor.map(_run_single_sim_cpu_thread, [weights_b_vec_np] * n_sims))
            print(f"✓ Threading completed ({len(results_a)} simulations).")
        else:
            print("Running serially (no parallelism)...")
            weights_a_vec_np = np.array([weights_a.get(col, 0.0) for col in ordered_cols], dtype=float)
            weights_b_vec_np = np.array([weights_b.get(col, 0.0) for col in ordered_cols], dtype=float)
            
            def _run_single_sim_cpu_serial(weights_vec_np):
                path = simulate_future(returns_matrix_np, weights_vec_np, years=years, block_size=block_size, xp_mod=np)
                return calculate_metrics(path)
            
            results_a = [_run_single_sim_cpu_serial(weights_a_vec_np) for _ in range(n_sims)]
            results_b = [_run_single_sim_cpu_serial(weights_b_vec_np) for _ in range(n_sims)]
            print(f"✓ Serial execution completed ({len(results_a)} simulations).")

    # 4. Analyze the "Compounding Safety"
    df_a = pd.DataFrame(results_a)
    df_b = pd.DataFrame(results_b)

    # Calculate Probability of Ruin (% of sims where final growth < 1.0)
    prob_ruin_a = (df_a['Growth'] < 1.0).sum() / len(df_a) * 100
    prob_ruin_b = (df_b['Growth'] < 1.0).sum() / len(df_b) * 100

    # Create summary statistics
    summary_data = {
        'Metric': [
            'CAGR (Mean)',
            'CAGR (StdDev)',
            'Volatility Annual (Mean)',
            'Sharpe Ratio (Mean)',
            'Sortino Ratio (Mean)',
            'Win Rate % (Mean)',
            'Positive Years % (Mean)',
            'Recovery Factor (Mean)',
            'Decade 1 CAGR (Mean)',
            'Decade 2 CAGR (Mean)',
            'Decade 3 CAGR (Mean)',
            'Probability of Ruin (%)',
            'Max Drawdown (Mean)',
            'Max Drawdown (StdDev)',
            'Total Growth (Mean)',
            'Total Growth (5th %ile)',
            'Growth $ from $10k (5th %ile)',
            'Martin Ratio (Mean)',
            'Efficiency Score (Mean)',
            f'Probability of {year_streak}-Year Underwater Streak',
            'Max Underwater Streak (Mean Days)',
            'Max Underwater Streak (Mean Years)',
            'Total Underwater Days'
        ],
        'A': [
            f"{df_a['CAGR'].mean():.2%}",
            f"{df_a['CAGR'].std():.2%}",
            f"{df_a['Volatility_Annual'].mean():.2%}",
            f"{df_a['Sharpe_Ratio'].mean():.3f}",
            f"{df_a['Sortino_Ratio'].mean():.3f}",
            f"{df_a['Win_Rate'].mean():.1f}%",
            f"{df_a['Positive_Years_Pct'].mean():.1f}%",
            f"{df_a['Recovery_Factor'].mean():.2f}x",
            f"{df_a['Decade1_CAGR'].mean():.2%}",
            f"{df_a['Decade2_CAGR'].mean():.2%}",
            f"{df_a['Decade3_CAGR'].mean():.2%}",
            f"{prob_ruin_a:.1f}%",
            f"{df_a['DD'].mean():.2%}",
            f"{df_a['DD'].std():.2%}",
            f"{df_a['Growth'].mean():.2f}x",
            f"{df_a['Growth'].quantile(0.05):.2f}x",
            f"${10000 * df_a['Growth'].quantile(0.05):,.0f}",
            f"{df_a['Martin_Ratio'].mean():.4f}",
            f"{df_a['Efficiency_Score'].mean():.4f}",
            f"{df_a[f'Has_{year_streak}Yr_Streak'].sum() / len(df_a) * 100:.1f}%",
            f"{df_a['Max_Underwater_Streak_Days'].mean():.0f}",
            f"{df_a['Max_Underwater_Streak_Days'].mean() / 252:.1f}",
            f"{df_a['Total_Underwater_Days'].mean():.0f}"
        ],
        'B': [
            f"{df_b['CAGR'].mean():.2%}",
            f"{df_b['CAGR'].std():.2%}",
            f"{df_b['Volatility_Annual'].mean():.2%}",
            f"{df_b['Sharpe_Ratio'].mean():.3f}",
            f"{df_b['Sortino_Ratio'].mean():.3f}",
            f"{df_b['Win_Rate'].mean():.1f}%",
            f"{df_b['Positive_Years_Pct'].mean():.1f}%",
            f"{df_b['Recovery_Factor'].mean():.2f}x",
            f"{df_b['Decade1_CAGR'].mean():.2%}",
            f"{df_b['Decade2_CAGR'].mean():.2%}",
            f"{df_b['Decade3_CAGR'].mean():.2%}",
            f"{prob_ruin_b:.1f}%",
            f"{df_b['DD'].mean():.2%}",
            f"{df_b['DD'].std():.2%}",
            f"{df_b['Growth'].mean():.2f}x",
            f"{df_b['Growth'].quantile(0.05):.2f}x",
            f"${10000 * df_b['Growth'].quantile(0.05):,.0f}",
            f"{df_b['Martin_Ratio'].mean():.4f}",
            f"{df_b['Efficiency_Score'].mean():.4f}",
            f"{df_b[f'Has_{year_streak}Yr_Streak'].sum() / len(df_b) * 100:.1f}%",
            f"{df_b['Max_Underwater_Streak_Days'].mean():.0f}",
            f"{df_b['Max_Underwater_Streak_Days'].mean() / 252:.1f}",
            f"{df_b['Total_Underwater_Days'].mean():.0f}"
        ]
    }

    summary_df = pd.DataFrame(summary_data)
    display(HTML("<h2>Compounding Safety Analysis (30-Year Monte Carlo Simulation)</h2>"))
    display(HTML(summary_df.to_html(index=False)))

[*********************100%***********************]  20 of 20 completed


Running with ThreadPoolExecutor (10 threads)...
✓ Threading completed (10000 simulations).


Metric,A,B
CAGR (Mean),16.15%,14.20%
CAGR (StdDev),3.12%,3.07%
Volatility Annual (Mean),16.32%,16.34%
Sharpe Ratio (Mean),0.838,0.718
Sortino Ratio (Mean),1.215,1.028
Win Rate % (Mean),53.6%,53.7%
Positive Years % (Mean),84.4%,81.5%
Recovery Factor (Mean),444.08x,248.53x
Decade 1 CAGR (Mean),16.21%,14.27%
Decade 2 CAGR (Mean),16.24%,14.28%


In [ ]:
import pandas as pd
import numpy as np

# 1. Define your 2026 Alpha Stack Weights
weights = {
    'VTI': 0.30,
    'DYNF': 0.30,
    'SPMO': 0.10,
    'AVUV': 0.10,
    'AVDV': 0.10,
    'VXUS': 0.10
}

# 2. Define your "Nightmare" and "Dream" Scenarios (Annualized Estimates)
# These aren't historical; they are your structural theses.
scenarios = {
    "Japan Nightmare (Stagnation)": {
        'VTI': -0.02, 'DYNF': 0.01, 'SPMO': -0.05, 'AVUV': 0.02, 'AVDV': 0.04, 'VXUS': 0.03
    },
    "Rentier Boom (Tech Monopoly)": {
        'VTI': 0.12, 'DYNF': 0.15, 'SPMO': 0.20, 'AVUV': 0.08, 'AVDV': 0.06, 'VXUS': 0.07
    },
    "Inflationary Reset (70s Style)": {
        'VTI': 0.01, 'DYNF': 0.03, 'SPMO': -0.08, 'AVUV': 0.07, 'AVDV': 0.06, 'VXUS': 0.04
    },
    "Global Bipolarity (US Crash / Intl Rally)": {
        'VTI': -0.05, 'DYNF': -0.02, 'SPMO': -0.10, 'AVUV': 0.01, 'AVDV': 0.08, 'VXUS': 0.06
    }
}

# 3. Calculate Portfolio Performance per Regime
results = {}
for name, returns in scenarios.items():
    port_return = sum(weights[ticker] * returns[ticker] for ticker in weights)
    results[name] = f"{port_return:.2%}"

# Output results
df_results = pd.DataFrame.from_dict(results, orient='index', columns=['Projected Annual Return'])
print(df_results)